In [7]:
from mysql.connector import connect
import pandas as pd

In [15]:
cnx = connect(
    user="admin",
    password="blaisesocial123", 
    host="blaisesocial-dev.cgv51hxv0wlh.us-east-1.rds.amazonaws.com",
    database="blaisesocial"
)
                  
        
cursor = cnx.cursor()
cursor.execute(
    f'SELECT * FROM pattern_detail;'

)
data = cursor.fetchall()
cursor.close()
cnx.close()

In [17]:
df = pd.DataFrame(data, columns=["RT_ID","SN","STOP_CD","PATTN_DETAIL_ID","IS_ACTIVE","LMT_SPD","DIST","TOPOLOGY"])
df

,RT_ID,SN,STOP_CD,PATTN_DETAIL_ID,IS_ACTIVE,LMT_SPD,DIST,TOPOLOGY
0,VBRCSMC-N1,9,URRACCA,Corredor Sur,1,60,10841,PA
1,VBRCSMC-N1,10,C39CT3A,Corredor Sur,1,60,11362,PA
2,VBRCSMC-N1,11,HNICT3A,Corredor Sur,1,60,11709,PA
3,VBRCSMC-N1,12,BOULECA,Corredor Sur,1,60,12228,PA
4,VBRCSMC-N1,13,MERMACA,Corredor Sur,1,60,13018,PA
...,...,...,...,...,...,...,...,...
63483,VVEROVR-N2,4,PAZTK1A,Ave. Boyd Roosevelt,1,60,1257,PA
63484,VVEROVR-N2,5,ERUTK1A,Ave. Boyd Roosevelt,1,60,1478,PA
63485,VVEROVR-N2,6,MASMITA,Ave. Boyd Roosevelt,1,60,2314,PA
63486,VVEROVR-N2,7,GESTATA,Ave. Boyd Roosevelt,1,60,3667,PA


In [19]:
df["STOP_CD"].shift(-1)

0        C39CT3A
1        HNICT3A
2        BOULECA
3        MERMACA
4        BALBO2A
          ...   
63483    ERUTK1A
63484    MASMITA
63485    GESTATA
63486    VILRICA
63487        NaN
Name: STOP_CD, Length: 63488, dtype: object

In [70]:
routes = df.sort_values(by=["RT_ID","SN"])

routes['STOP_CD_B'] = routes['STOP_CD']
routes['STOP_CD_E'] = routes['STOP_CD'].shift(-1)
routes['SN_B'] = routes['SN']
routes['SN_E'] = routes['SN'].shift(-1)

routes["DIST"] =  routes['DIST'].shift(-1) - routes['DIST'] 
# normalize last stops
routes.loc[(routes["DIST"].isna()) | (routes["DIST"]<0),"DIST"] = 0

routes["LMT_SPD"] =  routes['LMT_SPD'].shift(-1) 
routes.loc[routes["DIST"]==0,"LMT_SPD"] = 0
routes.loc[routes["DIST"]==0,"STOP_CD_E"] = routes["STOP_CD_B"]
routes.loc[routes["DIST"]==0,'SN_E'] = routes['SN_B']

routes =routes[["RT_ID",'SN_B',  "STOP_CD_B", 'SN_E',	"STOP_CD_E", 	"DIST", 	"LMT_SPD"]]
routes

,RT_ID,SN_B,STOP_CD_B,SN_E,STOP_CD_E,DIST,LMT_SPD
37497,12OMA,1,ESTAMAJ,2.0,MERMACA,558.0,60.0
37498,12OMA,2,MERMACA,3.0,C12CH3A,188.0,40.0
37499,12OMA,3,C12CH3A,4.0,CAHCH4A,199.0,30.0
37500,12OMA,4,CAHCH4A,5.0,EJMCH3A,250.0,30.0
37501,12OMA,5,EJMCH3A,6.0,PAQSANA,227.0,30.0
...,...,...,...,...,...,...,...
37492,VVVVEM-NA1,5,IGLDB3A,6.0,PCACE3A,288.0,60.0
37493,VVVVEM-NA1,6,PCACE3A,7.0,POLCE3A,458.0,40.0
37494,VVVVEM-NA1,7,POLCE3A,8.0,ERVCE3A,324.0,30.0
37495,VVVVEM-NA1,8,ERVCE3A,9.0,ESTAMAA,353.0,40.0


In [72]:
routes.to_csv("result.csv",index=False)